In [1]:
%matplotlib inline

# Forcing mechanisms for drifters entering and exiting Galveston Bay

An on-going area of interest for response efforts at the Texas General Land Office is under what conditions oil may exit or enter a given bay. Here, we limit our scope to Galveston Bay and use statistics to find relationships between drifter entrances/exits and potential forcing mechanisms.

## Relevant previous effort

This work builds on already-existing shelf model output (DJ's [20 year hindcast](http://barataria.tamu.edu:8080/thredds/dodsC/NcML/txla_hindcast_agg)) and Dongyu's effort to blended the "coarse" resolution Galveston Bay model with the shelf model to create a blended model product for seamlessly running drifters. The drifter simulations here are run using the blended model product.

## Run the drifters

The drifters were run on hafen (`/raid/home/kthyng/projects/bay/`) using the run.py script. Pertinent simulation details:

* Output drifter locations every 15 minutes
* Ran forward/backward for 14 days
* Circulation model output is hourly
* Surface drifters
* No subgrid diffusion
* 300 meter initial distance between drifters
* Drifters started in uniform array within bay
* Simulations are started from 4 dates:
 * 2010-02-01: winter winds, high river discharge
 * 2011-02-01: winter winds, low river discharge
 * 2010-07-01: summer winds, high river discharge
 * 2011-07-01: summer winds, low river discharge
* Simulations are run forward from the dates listed above, and backward from 2 weeks after the dates listed above. That is, the forward- and backward-running simulations are simulataneous but represent drifters exiting and entering the domain (respectively).
* New simulations are started every 4 hours for 2 weeks for a total number of simulations of: 6 * 14=84 simulations for each set (i.e. season and run direction)
* Each simulation has 13,340 drifters, so each set (i.e. season and run direction) has 13340 * 84=1,120,560 drifters or over 1.1 million.

Note that the appropriate file names for these simulations fit the pattern: `2010-02-01_forward_14days_dx300`.

The simulations have already been run using:

    python2 run.py > logs/[etc] &

The drifter tracks are stored on hafen in `tracks`.

The figure below shows the initial drifter locations.
![](figures/starting_points_bay_dx300.png)